In [1]:
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.providers.ibmq.managed import IBMQJobManager

# Loading your IBM Q account(s)
import qiskit
from qiskit import IBMQ
provider = IBMQ.load_account()

import numpy as np

import context
from device_independent_test import quantum_communicator
from device_independent_test import incompatible_measurement

# Using the QuantumDispatcher Class

A Dispatcher has 3 functions: run_and_transmit(), multi_run_and_transmit(), and batch_run_and_transmit(). 
Run_and_transmit takes in 5 parameters: input registers, output registers, a circuit of operations to prepare a transmitted state, a list operations to run on different devices, and the number of shots. Run_and_transmit() is intended to run a single job.
Multi_run_and_transmit takes in similar parameters. However, the preparation operations are now expected to be a list of operations. Similarly, the post operations are expected to be a list of lists. The ith element of each list corresponds to the circuit ran as the ith job. 
Batch_run_and_transmit creates and runs all permutations of operations given to it, both pre and post transmission. 

The cell below first creates a list of two operations to run to prepare states. Next, a list of two lists is created to define the operations desired after transmission. The first operation is the bb84 states and then Bob's measurement with y=0. The second is the bb84 states transmitted to Bob who measures when y=1. 
To create the dispatcher, a list of devices is required in the constructor. As this is a local dispatcher, only one device is required. Next, a multi run is executed and the counts are stored.

In [3]:
pre_ops = [incompatible_measurement.bb84_states(),incompatible_measurement.bb84_states()]
post_ops = [[QuantumCircuit(4),QuantumCircuit(4)],
            [incompatible_measurement.measure_circuit(0),incompatible_measurement.measure_circuit(1)]]

dispatch = quantum_communicator.LocalDispatcher([provider.get_backend('ibmq_qasm_simulator')])
counts = dispatch.multi_run_and_transmit("","",pre_ops,post_ops,1000)

In [4]:
print(counts)

[{'0000': 17, '0111': 3, '1011': 97, '1100': 12, '1001': 16, '1111': 17, '0100': 5, '0010': 93, '0110': 16, '0011': 24, '1000': 102, '1010': 491, '0001': 7, '1110': 99, '1101': 1}, {'0000': 19, '0111': 4, '1011': 99, '1100': 18, '1001': 533, '1111': 13, '0100': 3, '0101': 20, '0010': 4, '0011': 12, '1000': 95, '1010': 11, '0001': 81, '1110': 3, '1101': 85}]
